In [20]:
import pdfplumber
import pandas as pd
import re
from pathlib import Path

In [ ]:
pdf_dir = Path("../data/raw_pdfs")
bronze_dir = Path("../data/bronze")
bronze_dir.mkdir(exist_ok=True)

pdf_files = list(pdf_dir.glob("2024*.pdf"))
if not pdf_files:
    raise FileNotFoundError("No hay PDFs en raw_pdfs/")
pdf_file = pdf_files[0]

print(f"📄 Procesando: {pdf_file.name}")

📄 Procesando: 2025-10-03.pdf


In [22]:
rows = []
with pdfplumber.open(pdf_file) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if not text:
            continue

        # Dividir por líneas y limpiar
        lines = [re.sub(r'\s+', ' ', l.strip()) for l in text.split('\n') if l.strip()]

        for line in lines:
            # Saltar encabezados y metadatos
            if re.search(r'(nombre|presentación|unidad|valor|cal\.|análisis|analisis)', line, re.IGNORECASE):
                continue

            # Buscar líneas con precios
            if not re.search(r'\$\s?\d', line):
                continue

            # Buscar precios con regex
            precios = re.findall(r'\$\s?\d{1,3}(?:\.\d{3})*(?:,\d{2})?', line)

            if not precios:
                continue

            # Tomamos hasta 3 precios posibles
            precio_extra = precios[0] if len(precios) >= 1 else None
            precio_primera = precios[1] if len(precios) >= 2 else None
            valor_kilo = precios[2] if len(precios) >= 3 else None

            # Quitamos precios del texto base
            sin_precios = re.sub(r'\$\s?\d{1,3}(?:\.\d{3})*(?:,\d{2})?', '', line).strip()
            parts = sin_precios.split(" ")

            # Intentar extraer los campos antes de los precios
            producto = " ".join(parts[:-3]) if len(parts) > 3 else sin_precios
            presentacion = parts[-3] if len(parts) >= 3 else None
            cantidad = parts[-2] if len(parts) >= 2 else None
            unidad = parts[-1] if len(parts) >= 1 else None

            rows.append({
                "producto": producto.strip(),
                "presentacion": presentacion,
                "cantidad": cantidad,
                "unidad": unidad,
                "precio_calidad_extra": precio_extra,
                "precio_calidad_primera": precio_primera,
                "valor_x_kilo": valor_kilo,
            })

# Convertir a DataFrame
df = pd.DataFrame(rows)

# Limpieza de precios → número
for col in ["precio_calidad_extra", "precio_calidad_primera", "valor_x_kilo"]:
    df[col] = (
        df[col]
        .str.replace(r"[^\d,]", "", regex=True)
        .str.replace(",", "")
        .replace("", None)
        .astype(float)
    )

display(df.head(15))

,producto,presentacion,cantidad,unidad,precio_calidad_extra,precio_calidad_primera,valor_x_kilo
0,ALAS DE POLLO KILO 1 KILO 0,0,0,Estable,1600.0,1600.0,1600.0
1,MENUDENCIAS KILO 1 KILO 0,0,0,Estable,700.0,700.0,700.0
2,PECHUGA DE POLLO KILO 1 KILO 0,0,0,Estable,1750.0,1750.0,1750.0
3,PERNILES DE POLLO KILO 1 KILO 0,0,0,Estable,1500.0,1500.0,1500.0
4,POLLO SIN VICERAS KILO 1 KILO 0,0,0,Estable,2100.0,2100.0,2100.0
5,BAGRE DORADO KILO 1 KILO 0,0,0,Estable,2600.0,2600.0,2600.0
6,BAGRE PINTADO KILO 1 KILO 0,0,0,Estable,2400.0,2400.0,2400.0
7,BLANQUILLO GALLEGO KILO 1 KILO 0,0,0,Estable,1600.0,1600.0,1600.0
8,BOCA CHICO KILO 1 KILO 0,0,0,Estable,1760.0,1700.0,1760.0
9,CACHAMA KILO 1 KILO 0,0,0,Estable,1200.0,1200.0,1200.0


In [23]:
csv_path = bronze_dir / (pdf_file.stem + ".csv")
df.to_csv(csv_path, index=False)
print(f"💾 Guardado: {csv_path}")

💾 Guardado: ../data/bronze/2025-10-03.csv
